<a href="https://colab.research.google.com/github/41371122h-lichi/programming_group/blob/main/trip_starter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🌏 整合旅遊規劃系統 - 完整版

## 三大核心功能：
1. **✨ 自動生成行程** - MBTI 個性化 + 真實天氣 + 即時匯率
2. **🔧 行程優化器** - 風格/節奏調整 + PDF 編輯
3. **📍 地點推薦器** - 真實搜尋 + AI 推薦 + 地圖顯示

## 🆕 完整功能：
- ✅ **Gemini AI** - 智慧行程生成與分析
- ✅ **天氣 API** - 15天預報 + 70年歷史（Visual Crossing）
- ✅ **匯率 API** - 160+貨幣即時匯率（ExchangeRate）
- ✅ **地點搜尋** - Google Maps 真實資料（SerpAPI）
- ✅ **PDF 匯出** - 專業排版中文支援
- ✅ **互動地圖** - Folium 地圖顯示
- ✅ **智慧降級** - API 失敗時自動 fallback

---

## 🔑 API Keys 設定（必須先完成）

### 1. 點擊左側鑰匙圖示 🔑
### 2. 新增 4 個 Secrets：

| Secret Name | 用途 | 狀態 | 申請連結 |
|-------------|------|------|----------|
| `gemini` | AI 生成 | **必要** | https://aistudio.google.com/ |
| `serpapi` | 地點搜尋 | 建議 | https://serpapi.com/ |
| `weather_api` | 天氣資訊 | 建議 | https://www.visualcrossing.com/weather-api |
| `exchange_api` | 匯率換算 | 建議 | https://www.exchangerate-api.com/ |

### 3. 完成後執行下方所有 Cells

---

## 📦 步驟 1: 安裝系統套件（Colab 需要）

In [29]:
# 安裝 PDF 生成需要的系統套件
!apt-get update -qq
!apt-get install -y libpangocairo-1.0-0 libgdk-pixbuf2.0-0 libcairo2 fonts-noto-cjk

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fonts-noto-cjk is already the newest version (1:20220127+repack1-1).
libcairo2 is already the newest version (1.16.0-5ubuntu2).
libgdk-pixbuf2.0-0 is already the newest version (2.40.2-2build4).
libpangocairo-1.0-0 is already the newest version (1.50.6+ds-2ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 65 not upgraded.


## 📚 步驟 2: 安裝 Python 套件

In [30]:
!pip install -q gradio>=5.0.0 requests google-generativeai pypdf WeasyPrint markdown pandas geopy folium
!pip install google-search-results

## 🔧 步驟 3: 套件安裝檢查函數

這個函數會自動檢查並安裝缺少的套件

In [31]:
# 0. 安裝必要套件

import subprocess
import sys

def install_packages():
    """安裝所有必要的套件"""
    packages = [
        "gradio",
        "requests",
        "google-generativeai",
        "pypdf",
        "WeasyPrint",
        "markdown",
        "pandas",
        "geopy",
        "folium",
    ]

    for package in packages:
        try:
            __import__(package.replace('-', '_').split('[')[0])
        except ImportError:
            print(f"Installing {package}...")
            subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# 嘗試安裝套件
try:
    install_packages()
except Exception as e:
    print(f"Warning: Could not install all packages: {e}")


from datetime import datetime, timedelta
import os
import requests

Installing google-generativeai...
Installing WeasyPrint...


## 📦 步驟 4: 導入所有套件

導入並檢查所有必要的 Python 套件

In [32]:
# 1. 導入套件

# google-generativeai
try:
    import google.generativeai as genai
    from google.generativeai.types import HarmCategory, HarmBlockThreshold
    GENAI_AVAILABLE = True
except ImportError:
    GENAI_AVAILABLE = False
    print("Warning: google-generativeai not available")

# PDF / HTML / Markdown
try:
    from pypdf import PdfReader
    from weasyprint import HTML, CSS
    from weasyprint.text.fonts import FontConfiguration
    import markdown
    WEASYPRINT_AVAILABLE = True
except ImportError:
    WEASYPRINT_AVAILABLE = False
    print("Warning: WeasyPrint not available")

# pandas / geopy / folium
try:
    import pandas as pd
    from geopy.geocoders import Nominatim
    import folium
    GEOPY_AVAILABLE = True
except ImportError:
    GEOPY_AVAILABLE = False
    print("Warning: geopy/folium not available")

import json
import requests
from datetime import datetime, timedelta

import gradio as gr
import markdown
from weasyprint import HTML
import json
import pandas as pd

## 🔑 步驟 5: API Keys 設定與驗證

系統會自動從 Colab Secrets 讀取 API Keys

In [33]:
# 2. API 設定

import os

GEMINI_API_KEY = None

# 嘗試從 Colab Secrets 讀取
try:
    from google.colab import userdata
    GEMINI_API_KEY = userdata.get('gemini')
    print("✅ 成功從 Colab Secrets 讀取 API Key")
except:
    # 不在 Colab 時，改從環境變數讀取
    GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')
    if GEMINI_API_KEY:
        print("✅ 從環境變數讀取 API Key")
    else:
        print("⚠️ 未找到 Gemini API Key")

# 設定 Gemini API
model = None
gemini_client = None

if GENAI_AVAILABLE and GEMINI_API_KEY:
    try:
        genai.configure(api_key=GEMINI_API_KEY)
        model = genai.GenerativeModel('gemini-2.0-flash')  # 使用穩定版
        gemini_client = genai
        print("✅ Gemini API 已成功配置！")
    except Exception as e:
        print(f"❌ API 配置失敗: {e}")
        GENAI_AVAILABLE = False

# Google Search API（地點推薦）
SERPAPI_KEY = os.getenv('SERPAPI_KEY', '')

# Weather API（天氣資訊）
WEATHER_API_KEY = os.getenv('WEATHER_API_KEY', '')

# Exchange Rate API（匯率轉換）
EXCHANGE_API_KEY = os.getenv('EXCHANGE_API_KEY', '')

# 如果在 Colab，補讀取 Secrets
if not SERPAPI_KEY or not WEATHER_API_KEY or not EXCHANGE_API_KEY:
    try:
        from google.colab import userdata

        if not SERPAPI_KEY:
            SERPAPI_KEY = userdata.get('serpapi')
            if SERPAPI_KEY:
                print("✅ 成功從 Colab Secrets 讀取 SerpAPI Key")

        if not WEATHER_API_KEY:
            WEATHER_API_KEY = userdata.get('weather_api')
            if WEATHER_API_KEY:
                print("✅ 成功從 Colab Secrets 讀取 Weather API Key")

        if not EXCHANGE_API_KEY:
            EXCHANGE_API_KEY = userdata.get('exchange_api')
            if EXCHANGE_API_KEY:
                print("✅ 成功從 Colab Secrets 讀取 Exchange Rate API Key")

    except:
        pass

# 未設定的 API Key 提醒
if not SERPAPI_KEY:
    print("⚠️ 未設定 SerpAPI Key，地點推薦功能將使用範例資料")

if not WEATHER_API_KEY:
    print("⚠️ 未設定 Weather API Key，將使用預設天氣資訊或歷史資料參考")

if not EXCHANGE_API_KEY:
    print("⚠️ 未設定 Exchange Rate API Key，將無法提供匯率換算")



✅ 成功從 Colab Secrets 讀取 API Key
✅ Gemini API 已成功配置！
✅ 成功從 Colab Secrets 讀取 SerpAPI Key
✅ 成功從 Colab Secrets 讀取 Weather API Key
✅ 成功從 Colab Secrets 讀取 Exchange Rate API Key


## ⭐ 功能 1-1: 問卷

In [34]:
# ==========================================
# 功能1:旅客類型問卷系統
# ==========================================

# 旅客類型定義
TRAVELER_TYPES = {
    "探險型": {
        "description": "喜歡刺激活動、極限運動、冒險體驗",
        "keywords": ["戶外冒險", "極限運動", "登山健行", "水上活動", "刺激體驗"],
        "avoid": ["行程太緩慢", "太多購物時間"],
        "pace": "緊湊充實",
        "budget_focus": "活動門票與裝備",
        "icon": "🏔️"
    },
    "文化型": {
        "description": "熱愛博物館、歷史遺跡、藝術展覽",
        "keywords": ["博物館", "古蹟", "藝術展", "文化體驗", "歷史景點"],
        "avoid": ["過於商業化", "缺乏深度"],
        "pace": "適中從容",
        "budget_focus": "門票與文化導覽",
        "icon": "🏛️"
    },
    "美食型": {
        "description": "以品嚐當地美食為主要旅遊目的",
        "keywords": ["當地美食", "米其林餐廳", "傳統小吃", "市場探索", "料理體驗"],
        "avoid": ["用餐時間太趕", "餐廳選擇受限"],
        "pace": "彈性悠閒",
        "budget_focus": "餐飲與美食體驗",
        "icon": "🍜"
    },
    "購物型": {
        "description": "享受逛街購物、精品採購",
        "keywords": ["購物商圈", "outlet", "精品店", "市集", "紀念品"],
        "avoid": ["購物時間不足", "沒有行李空間"],
        "pace": "彈性自由",
        "budget_focus": "購物與血拼",
        "icon": "🛍️"
    },
    "放鬆型": {
        "description": "偏好慢步調、咖啡廳、SPA、海灘度假",
        "keywords": ["海灘度假", "SPA", "咖啡廳", "慢活", "放空"],
        "avoid": ["行程太緊湊", "每天換飯店"],
        "pace": "緩慢悠閒",
        "budget_focus": "住宿與休閒設施",
        "icon": "🏖️"
    },
    "自然型": {
        "description": "喜歡戶外活動、生態旅遊、親近大自然",
        "keywords": ["國家公園", "生態導覽", "賞鳥", "森林浴", "自然景觀"],
        "avoid": ["都市停留太久", "室內行程過多"],
        "pace": "適中靈活",
        "budget_focus": "戶外活動與生態體驗",
        "icon": "🌲"
    },
    "社交型": {
        "description": "重視夜生活、派對、結識新朋友",
        "keywords": ["夜生活", "酒吧", "音樂節", "派對", "社交活動"],
        "avoid": ["太早結束行程", "缺乏社交機會"],
        "pace": "晚間為主",
        "budget_focus": "娛樂與夜間活動",
        "icon": "🎉"
    },
    "親子型": {
        "description": "適合全家出遊,重視安全與親子設施",
        "keywords": ["親子景點", "遊樂園", "動物園", "互動體驗", "教育意義"],
        "avoid": ["行程太累", "不適合小孩"],
        "pace": "輕鬆適中",
        "budget_focus": "親子活動與家庭套票",
        "icon": "👨‍👩‍👧‍👦"
    }
}

# 問卷題目設計
QUESTIONNAIRE = {
    "q1": {
        "question": "旅遊時你最期待什麼?(可複選,最多3個)",
        "options": {
            "刺激冒險體驗": ["探險型", "自然型"],
            "深度文化探索": ["文化型"],
            "品嚐在地美食": ["美食型"],
            "血拼購物": ["購物型"],
            "放鬆身心": ["放鬆型"],
            "親近大自然": ["自然型", "探險型"],
            "體驗夜生活": ["社交型"],
            "親子同樂": ["親子型"]
        }
    },
    "q2": {
        "question": "你偏好的行程節奏是?",
        "options": {
            "緊湊充實,每天多個景點": ["探險型", "文化型"],
            "適中平衡,有景點也有休息": ["美食型", "自然型", "親子型"],
            "彈性自由,隨興安排": ["購物型", "社交型"],
            "緩慢悠閒,深度停留": ["放鬆型"]
        }
    },
    "q3": {
        "question": "同行旅伴類型?",
        "options": {
            "獨自旅行": ["探險型", "文化型", "自然型"],
            "情侶/夫妻": ["放鬆型", "美食型"],
            "家庭親子": ["親子型"],
            "朋友團體": ["社交型", "購物型", "探險型"]
        }
    },
    "q4": {
        "question": "你喜歡的活動強度?",
        "options": {
            "高強度(登山、極限運動等)": ["探險型"],
            "中等強度(健行、單車等)": ["自然型", "文化型"],
            "輕度活動(散步、參觀等)": ["美食型", "購物型", "親子型"],
            "低強度(度假、休息為主)": ["放鬆型"]
        }
    },
    "q5": {
        "question": "旅遊預算主要想花在哪?(選2個)",
        "options": {
            "特色活動體驗": ["探險型", "文化型"],
            "高級餐廳美食": ["美食型"],
            "購物與紀念品": ["購物型"],
            "舒適住宿": ["放鬆型"],
            "交通移動": ["自然型"],
            "夜間娛樂": ["社交型"],
            "親子設施門票": ["親子型"]
        }
    }
}


def analyze_questionnaire(answers):
    """
    分析問卷答案,推薦旅客類型

    Args:
        answers: dict, 格式 {"q1": [...], "q2": "...", ...}

    Returns:
        dict: {
            "primary_type": "主要類型",
            "secondary_type": "次要類型",
            "scores": {"探險型": 5, "文化型": 3, ...}
        }
    """

    # 計分
    type_scores = {t: 0 for t in TRAVELER_TYPES.keys()}

    for q_id, answer in answers.items():
        question_data = QUESTIONNAIRE.get(q_id, {})
        options = question_data.get("options", {})

        # 處理單選
        if isinstance(answer, str):
            types = options.get(answer, [])
            for t in types:
                type_scores[t] += 1

        # 處理多選
        elif isinstance(answer, list):
            for ans in answer:
                types = options.get(ans, [])
                for t in types:
                    type_scores[t] += 1

    # 排序
    sorted_types = sorted(type_scores.items(), key=lambda x: x[1], reverse=True)

    primary_type = sorted_types[0][0] if sorted_types[0][1] > 0 else "美食型"
    secondary_type = sorted_types[1][0] if len(sorted_types) > 1 and sorted_types[1][1] > 0 else None

    return {
        "primary_type": primary_type,
        "secondary_type": secondary_type,
        "scores": type_scores
    }


def format_type_result(analysis_result):
    """格式化類型分析結果"""

    primary = analysis_result["primary_type"]
    secondary = analysis_result["secondary_type"]

    primary_info = TRAVELER_TYPES[primary]

    result = f"""
## {primary_info['icon']} 你的旅客類型:{primary}

**特質描述:** {primary_info['description']}

**推薦活動:** {', '.join(primary_info['keywords'][:3])}

**行程節奏:** {primary_info['pace']}

**預算建議:** 重點投資在{primary_info['budget_focus']}
"""

    if secondary:
        secondary_info = TRAVELER_TYPES[secondary]
        result += f"\n**次要類型:** {secondary_info['icon']} {secondary} - {secondary_info['description']}"

    return result

# 問卷分析邏輯
def process_questionnaire(q1, q2, q3, q4, q5):
    """處理問卷並返回分析結果"""

    # 驗證 Q1 (最多3個)
    if len(q1) > 3:
        return "❌ 問題1請最多選擇3個選項!", None
    if len(q1) == 0:
        return "❌ 問題1請至少選擇1個選項!", None

    # 驗證 Q5 (恰好2個)
    if len(q5) != 2:
        return "❌ 問題5請選擇恰好2個選項!", None

    # 驗證其他必填
    if not q2 or not q3 or not q4:
        return "❌ 請回答所有問題!", None

    # 組織答案
    answers = {
        "q1": q1,
        "q2": q2,
        "q3": q3,
        "q4": q4,
        "q5": q5
    }

    # 分析
    analysis = analyze_questionnaire(answers)
    result_text = format_type_result(analysis)

    return result_text, analysis


## ⭐ 功能 1-2: 天氣資訊系統

整合 Visual Crossing Weather API：
- 🌤️ 15 天天氣預報
- 📊 70 年歷史資料參考
- 🔄 智慧降級機制

In [35]:
def get_weather_info(country, city, start_date, num_days):
    """
    獲取天氣資訊

    Args:
        country: 國家名稱
        city: 城市名稱
        start_date: 出發日期 (YYYY-MM-DD)
        num_days: 天數

    Returns:
        dict: {
            "success": bool,
            "data_source": "real_api" | "fallback",
            "weather_data": [
                {
                    "date": "2025-06-01",
                    "temp_min": 18,
                    "temp_max": 25,
                    "temp_display": "18-25°C",
                    "condition": "晴朗",
                    "condition_en": "Clear",
                    "humidity": 65,
                    "suggestion": "適合戶外活動"
                },
                ...
            ],
            "error_message": "" (如果有錯誤)
        }
    """

    # 驗證日期格式
    try:
        start_dt = datetime.strptime(start_date, "%Y-%m-%d")
    except ValueError:
        return {
            "success": False,
            "data_source": "error",
            "weather_data": [],
            "error_message": "日期格式錯誤,請使用 YYYY-MM-DD 格式"
        }

    # 如果沒有 API Key,使用預設資料
    if not WEATHER_API_KEY:
        print("⚠️ 未設定 Weather API Key,使用預設天氣資訊")
        return _get_fallback_weather(start_date, num_days)

    # 嘗試從 Visual Crossing Weather API 獲取真實數據
    try:
        end_date = (start_dt + timedelta(days=num_days - 1)).strftime("%Y-%m-%d")

        url = (
            f"https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/"
            f"{city},{country}/{start_date}/{end_date}"
            f"?unitGroup=metric&key={WEATHER_API_KEY}&contentType=json&lang=zh"
        )

        response = requests.get(url, timeout=15)

        # 檢查 HTTP 狀態碼
        if response.status_code == 400:
            print(f"❌ Weather API 請求錯誤: 城市或日期參數無效")
            return _get_fallback_weather(start_date, num_days,
                                        error="城市名稱或日期範圍無效")

        elif response.status_code == 401:
            print(f"❌ Weather API Key 無效或已過期")
            return _get_fallback_weather(start_date, num_days,
                                        error="API Key 無效")

        elif response.status_code == 429:
            print(f"❌ Weather API 請求次數超過限制")
            return _get_fallback_weather(start_date, num_days,
                                        error="API 請求次數超限")

        elif response.status_code != 200:
            print(f"❌ Weather API 回應錯誤: HTTP {response.status_code}")
            return _get_fallback_weather(start_date, num_days,
                                        error=f"HTTP {response.status_code}")

        # 解析 JSON
        data = response.json()

        if "days" not in data or len(data["days"]) == 0:
            print(f"❌ Weather API 回應中沒有天氣數據")
            return _get_fallback_weather(start_date, num_days,
                                        error="無天氣數據")

        # 處理真實天氣數據
        weather_data = []
        for day in data["days"]:
            condition_en = day.get("conditions", "Unknown")
            condition_zh = _translate_condition(condition_en)

            weather_data.append({
                "date": day.get("datetime"),
                "temp_min": round(day.get("tempmin", 20), 1),
                "temp_max": round(day.get("tempmax", 25), 1),
                "temp_display": f"{round(day.get('tempmin', 20), 1)}-{round(day.get('tempmax', 25), 1)}°C",
                "condition": condition_zh,
                "condition_en": condition_en,
                "humidity": day.get("humidity", 60),
                "suggestion": _get_weather_suggestion(condition_en, day.get("tempmax", 25))
            })

        return {
            "success": True,
            "data_source": "real_api",
            "weather_data": weather_data,
            "error_message": ""
        }

    except requests.exceptions.Timeout:
        print(f"❌ Weather API 請求超時")
        return _get_fallback_weather(start_date, num_days, error="請求超時")

    except requests.exceptions.ConnectionError:
        print(f"❌ Weather API 連線失敗")
        return _get_fallback_weather(start_date, num_days, error="網路連線失敗")

    except Exception as e:
        print(f"❌ Weather API 取得失敗: {str(e)}")
        return _get_fallback_weather(start_date, num_days, error=str(e))


def _get_fallback_weather(start_date, num_days, error=""):
    """
    產生預設天氣資料 (當 API 失敗時使用)

    Args:
        start_date: 開始日期
        num_days: 天數
        error: 錯誤訊息

    Returns:
        dict: 包含預設天氣資料的字典
    """
    weather_data = []

    # 根據月份給出較合理的預設溫度
    try:
        start_dt = datetime.strptime(start_date, "%Y-%m-%d")
        month = start_dt.month
    except:
        month = 6  # 預設6月

    # 季節性溫度調整
    if month in [12, 1, 2]:  # 冬季
        temp_range = (10, 18)
        condition = "多雲"
    elif month in [3, 4, 5]:  # 春季
        temp_range = (15, 23)
        condition = "晴朗"
    elif month in [6, 7, 8]:  # 夏季
        temp_range = (22, 30)
        condition = "晴朗偶有陣雨"
    else:  # 秋季
        temp_range = (16, 24)
        condition = "晴朗"

    for i in range(num_days):
        try:
            date = (datetime.strptime(start_date, "%Y-%m-%d") + timedelta(days=i)).strftime("%Y-%m-%d")
        except:
            date = start_date

        weather_data.append({
            "date": date,
            "temp_min": temp_range[0],
            "temp_max": temp_range[1],
            "temp_display": f"{temp_range[0]}-{temp_range[1]}°C",
            "condition": condition,
            "condition_en": "Clear",
            "humidity": 65,
            "suggestion": "天氣資訊僅供參考"
        })

    return {
        "success": False,
        "data_source": "fallback",
        "weather_data": weather_data,
        "error_message": error if error else "未設定 API Key"
    }


def _translate_condition(condition_en):
    """將英文天氣狀況翻譯成中文"""

    translation_map = {
        "Clear": "晴朗",
        "Partially cloudy": "部分多雲",
        "Overcast": "陰天",
        "Rain": "雨天",
        "Rain, Overcast": "陰雨",
        "Rain, Partially cloudy": "多雲有雨",
        "Snow": "下雪",
        "Snow, Overcast": "陰天下雪",
        "Thunderstorm": "雷雨",
        "Fog": "霧",
        "Wind": "強風",
        "Cloudy": "多雲"
    }

    # 模糊匹配
    condition_lower = condition_en.lower()

    if "clear" in condition_lower or "sunny" in condition_lower:
        return "晴朗"
    elif "rain" in condition_lower and "thunder" in condition_lower:
        return "雷雨"
    elif "rain" in condition_lower:
        return "雨天"
    elif "snow" in condition_lower:
        return "下雪"
    elif "cloud" in condition_lower or "overcast" in condition_lower:
        return "多雲"
    elif "fog" in condition_lower:
        return "霧"
    elif "wind" in condition_lower:
        return "強風"
    else:
        return translation_map.get(condition_en, condition_en)


def _get_weather_suggestion(condition_en, temp_max):
    """根據天氣狀況給出建議"""

    condition_lower = condition_en.lower()

    if "rain" in condition_lower or "storm" in condition_lower:
        return "記得攜帶雨具"
    elif "snow" in condition_lower:
        return "注意保暖,穿著防滑鞋"
    elif temp_max > 30:
        return "天氣炎熱,注意防曬與補充水分"
    elif temp_max < 10:
        return "天氣寒冷,建議多層次穿搭"
    elif "clear" in condition_lower or "sunny" in condition_lower:
        return "適合戶外活動"
    else:
        return "適合旅遊"


def format_weather_display(weather_result):
    """
    格式化天氣資訊為 Markdown 顯示

    Args:
        weather_result: get_weather_info() 的返回結果

    Returns:
        str: Markdown 格式的天氣資訊
    """

    if not weather_result["success"]:
        output = f"""
### ⚠️ 天氣資訊 (預設數據)

> **數據來源:** 預設天氣資料 (僅供參考)
> **原因:** {weather_result['error_message']}

"""
    else:
        output = f"""
### ☀️ 天氣預報 (真實數據)

> **數據來源:** Visual Crossing Weather API
> **更新時間:** {datetime.now().strftime("%Y-%m-%d %H:%M")}

"""

    # 天氣列表
    for day in weather_result["weather_data"]:
        output += f"""
**{day['date']}**
🌡️ 溫度: {day['temp_display']} | 💧 濕度: {day['humidity']}%
☁️ 天氣: {day['condition']} | 💡 {day['suggestion']}

"""

    return output


## ⭐ 功能 1-3: 匯率換算系統

整合 ExchangeRate API：
- 💱 160+ 種貨幣支援
- 💰 即時匯率查詢
- 📊 智慧預算建議

In [36]:
# ==========================================
# 功能3:匯率資訊獲取 (極簡版)
# ==========================================

import os
import requests

# 讀取 API Key
EXCHANGE_API_KEY = os.getenv("EXCHANGE_API_KEY", "")

if not EXCHANGE_API_KEY:
    try:
        from google.colab import userdata
        EXCHANGE_API_KEY = userdata.get("exchange_api")
    except:
        pass


# 國家名稱 → 貨幣代碼對應表
COUNTRY_TO_CURRENCY = {
    # 亞洲
    "日本": "JPY", "韓國": "KRW", "南韓": "KRW",
    "泰國": "THB", "新加坡": "SGD", "馬來西亞": "MYR",
    "越南": "VND", "印尼": "IDR", "菲律賓": "PHP",
    "中國": "CNY", "香港": "HKD", "澳門": "MOP",
    "台灣": "TWD", "印度": "INR",

    # 歐美
    "美國": "USD", "英國": "GBP",
    "歐洲": "EUR", "法國": "EUR", "德國": "EUR", "義大利": "EUR", "西班牙": "EUR",
    "瑞士": "CHF", "瑞典": "SEK", "挪威": "NOK", "丹麥": "DKK",
    "澳洲": "AUD", "紐西蘭": "NZD", "加拿大": "CAD",

    # 其他
    "土耳其": "TRY", "以色列": "ILS", "阿聯酋": "AED", "阿聯": "AED",
    "墨西哥": "MXN", "巴西": "BRL", "南非": "ZAR",
    "冰島": "ISK", "波蘭": "PLN", "捷克": "CZK", "匈牙利": "HUF",
}


# 預設匯率 (2025年1月參考值)
DEFAULT_RATES = {
    "USD": 0.032, "EUR": 0.029, "GBP": 0.025,
    "JPY": 4.5, "KRW": 42.0, "CNY": 0.23,
    "HKD": 0.25, "SGD": 0.042, "THB": 1.1,
    "MYR": 0.14, "VND": 790, "IDR": 500, "PHP": 1.8,
    "AUD": 0.048, "NZD": 0.052, "CAD": 0.044,
    "CHF": 0.028, "SEK": 0.34, "NOK": 0.34, "DKK": 0.22,
    "INR": 2.7, "AED": 0.117, "TRY": 0.95,
    "MXN": 0.55, "BRL": 0.16, "ZAR": 0.58,
    "ISK": 0.22, "PLN": 0.0075, "CZK": 0.0013, "HUF": 0.0085,
}


def get_exchange_rate(country_name):
    """
    取得匯率 (從 TWD 到該國貨幣)

    Args:
        country_name: 國家名稱 (例如: "日本")

    Returns:
        dict: {
            "success": bool,           # API 是否成功
            "country": "日本",
            "currency": "JPY",
            "rate": 4.5,              # 1 TWD = 4.5 JPY
            "source": "api" | "default"
        }
    """

    # 步驟1: 國家名 → 貨幣代碼
    currency_code = COUNTRY_TO_CURRENCY.get(country_name, "USD")

    # 如果是台灣,直接返回 1:1
    if currency_code == "TWD":
        return {
            "success": True,
            "country": country_name,
            "currency": "TWD",
            "rate": 1.0,
            "source": "same_currency"
        }

    # 步驟2: 嘗試從 API 取得匯率
    if EXCHANGE_API_KEY:
        try:
            url = f"https://v6.exchangerate-api.com/v6/{EXCHANGE_API_KEY}/latest/TWD"
            response = requests.get(url, timeout=10)

            if response.status_code == 200:
                data = response.json()

                if data.get("result") == "success":
                    rate = data["conversion_rates"].get(currency_code)

                    if rate:
                        return {
                            "success": True,
                            "country": country_name,
                            "currency": currency_code,
                            "rate": round(rate, 4),
                            "source": "api"
                        }
        except:
            pass  # API 失敗,繼續使用預設值

    # 步驟3: 使用預設匯率
    default_rate = DEFAULT_RATES.get(currency_code, 1.0)

    return {
        "success": False,
        "country": country_name,
        "currency": currency_code,
        "rate": default_rate,
        "source": "default"
    }


def format_exchange_info(country, budget_range):
    """
    格式化匯率資訊為 Markdown

    Args:
        country: 國家名稱
        budget_range: 預算範圍 ("經濟實惠" | "中等預算" | "高端奢華")

    Returns:
        str: Markdown 格式的匯率資訊
    """

    # 取得匯率
    rate_info = get_exchange_rate(country)

    # 如果是台灣,不顯示匯率資訊
    if rate_info["source"] == "same_currency":
        return ""

    currency = rate_info["currency"]
    rate = rate_info["rate"]
    is_real = rate_info["success"]

    # 預算範圍對應的金額
    budget_amounts = {
        "經濟實惠": [1000, 2000, 3000],
        "中等預算": [3000, 5000, 8000],
        "高端奢華": [8000, 15000, 25000],
    }

    amounts_twd = budget_amounts.get(budget_range, [1000, 3000, 5000])

    # 計算外幣金額
    amounts_foreign = [round(twd * rate, 2) for twd in amounts_twd]

    # 格式化輸出
    if is_real:
        result = f"""
### 💱 匯率資訊 (真實數據)

> **匯率:** 1 TWD = {rate} {currency}
> **數據來源:** ExchangeRate-API

"""
    else:
        result = f"""
### 💱 匯率資訊 (參考數據)

> **匯率:** 1 TWD ≈ {rate} {currency}
> **數據來源:** 預設匯率 (僅供參考)

"""

    # 預算參考
    result += f"**{budget_range}預算參考:**\n\n"

    for twd, foreign in zip(amounts_twd, amounts_foreign):
        # 格式化外幣金額
        if foreign >= 10000:
            foreign_str = f"{foreign:,.0f}"
        elif foreign >= 100:
            foreign_str = f"{foreign:,.1f}"
        else:
            foreign_str = f"{foreign:.2f}"

        result += f"- {twd:,} TWD → **{foreign_str} {currency}**\n"

    return result


def calculate_amount(twd_amount, country):
    """
    計算特定金額的外幣兌換

    Args:
        twd_amount: 台幣金額
        country: 國家名稱

    Returns:
        dict: {
            "twd": 5000,
            "currency": "JPY",
            "amount": 22500,
            "rate": 4.5
        }
    """

    rate_info = get_exchange_rate(country)

    return {
        "twd": twd_amount,
        "currency": rate_info["currency"],
        "amount": round(twd_amount * rate_info["rate"], 2),
        "rate": rate_info["rate"]
    }


## ⭐ 功能 1-4: AI 行程生成

整合所有資訊生成完整行程：
- 🎭 MBTI 個性化分析（16種類型）
- 🌤️ 真實天氣資訊
- 💱 即時匯率換算
- 📄 PDF 匯出功能

In [37]:
def generate_itinerary(country, city, start_date, num_days, traveler_type, budget, interests=""):
    """
    生成旅遊行程

    Args:
        country: 國家名稱
        city: 城市名稱
        start_date: 出發日期 (YYYY-MM-DD)
        num_days: 天數
        traveler_type: 旅客類型 (例如: "美食型")
        budget: 預算範圍 ("經濟實惠" | "中等預算" | "高端奢華")
        interests: 額外興趣 (選填)

    Returns:
        tuple: (行程文字, PDF檔案路徑)
    """

    # 檢查 Gemini 模型
    if not model:
        return "❌ Gemini API 未配置,請設定 GEMINI_API_KEY", None

    # 驗證日期格式
    try:
        datetime.strptime(start_date, "%Y-%m-%d")
    except ValueError:
        return "❌ 日期格式錯誤,請使用 YYYY-MM-DD 格式", None

    # 驗證天數
    if num_days < 1 or num_days > 14:
        return "❌ 天數必須在 1-14 天之間", None

    try:
        # ========== 1. 取得旅客類型資訊 ==========
        traveler_info = TRAVELER_TYPES.get(traveler_type, TRAVELER_TYPES["美食型"])

        # ========== 2. 取得天氣資訊 ==========
        weather_result = get_weather_info(country, city, start_date, num_days)

        # 格式化天氣資訊
        if weather_result["success"]:
            weather_section = f"""
**天氣預報 (真實數據):**
來源: Visual Crossing Weather API
"""
            for day in weather_result["weather_data"]:
                weather_section += f"""
- {day['date']}: {day['condition']} ({day['temp_display']}), {day['suggestion']}
"""
        else:
            weather_section = f"""
**天氣參考 (預設數據,僅供參考):**
原因: {weather_result['error_message']}
"""
            for day in weather_result["weather_data"]:
                weather_section += f"""
- {day['date']}: {day['condition']} ({day['temp_display']})
"""

        # ========== 3. 取得匯率資訊 ==========
        exchange_markdown = format_exchange_info(country, budget)

        # 取得匯率詳細資訊 (用於 prompt)
        rate_info = get_exchange_rate(country)

        if rate_info["source"] == "same_currency":
            exchange_section = ""
        elif rate_info["success"]:
            exchange_section = f"""
**匯率資訊 (真實數據):**
- 1 TWD = {rate_info['rate']} {rate_info['currency']}
- 來源: ExchangeRate-API
"""
        else:
            exchange_section = f"""
**匯率參考 (預設數據,僅供參考):**
- 1 TWD ≈ {rate_info['rate']} {rate_info['currency']}
"""

        # ========== 4. 建構 Prompt ==========
        prompt = f"""
你是一位專業的旅遊規劃師,請為以下旅客規劃 {num_days} 天的詳細旅遊行程。

**旅客資訊:**
- 目的地: {country} {city}
- 出發日期: {start_date}
- 旅遊天數: {num_days} 天
- 旅客類型: {traveler_type} {traveler_info['icon']}
- 預算範圍: {budget}
{f"- 特殊興趣: {interests}" if interests else ""}

**旅客類型特徵:**
- 描述: {traveler_info['description']}
- 偏好活動: {', '.join(traveler_info['keywords'])}
- 行程節奏: {traveler_info['pace']}
- 預算重點: {traveler_info['budget_focus']}
- 避免事項: {', '.join(traveler_info['avoid'])}

{weather_section}

{exchange_section}

**請提供以下內容:**

1. **行程總覽**
   - 簡短介紹這趟旅程的亮點
   - 根據旅客類型說明為什麼這個行程適合他們

2. **每日詳細行程** (共 {num_days} 天)
   每天包含:
   - 早上、中午、下午、晚上的活動安排
   - 推薦景點 (符合旅客類型偏好)
   - 餐廳建議 (含預算參考)
   - 交通方式
   - 根據天氣給出貼心提醒

3. **預算規劃**
   - 住宿建議 (符合預算範圍)
   - 交通預算估算
   - 餐飲預算估算
   - 活動門票預算
   - 總預算區間 (TWD 和當地貨幣)

4. **實用資訊**
   - 交通卡/套票建議
   - 必備 App 推薦
   - 注意事項

5. **行李準備建議**
   - 根據天氣提醒該帶的衣物
   - 根據旅客類型推薦的裝備

**重要提醒:**
- 行程安排必須符合「{traveler_type}」的特質
- 避免安排旅客不喜歡的活動類型: {', '.join(traveler_info['avoid'])}
- 行程節奏要符合「{traveler_info['pace']}」的要求
- 預算分配重點放在「{traveler_info['budget_focus']}」
- 如果天氣或匯率資訊標示為「預設數據」,請提醒旅客行前再次確認

請用 Markdown 格式輸出,使用清晰的標題和列表。請勿出現表情符號。
"""

        # ========== 5. 呼叫 Gemini API ==========
        response = model.generate_content(
            prompt,
            safety_settings={
                HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
                HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
                HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
                HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
            },
        )

        itinerary_text = response.text

        # 在行程前面加上天氣和匯率資訊
        full_itinerary = f"""# {country} {city} {num_days}日遊行程

{exchange_markdown}

{format_weather_display(weather_result)}

---

{itinerary_text}
"""

        # ========== 6. 生成 PDF ==========
        pdf_path = None

        if WEASYPRINT_AVAILABLE:
            try:
                # 轉換為 HTML
                html_content = markdown.markdown(
                    full_itinerary,
                    extensions=['tables', 'fenced_code']
                )

                # 完整的 HTML (含中文字體)
                html_full = f"""
                <html>
                <head>
                    <meta charset="utf-8">
                    <style>
                        @import url('https://fonts.googleapis.com/css2?family=Noto+Sans+TC:wght@400;700&display=swap');

                        body {{
                            font-family: 'Noto Sans TC', 'Microsoft JhengHei', sans-serif;
                            line-height: 1.8;
                            padding: 40px;
                            color: #333;
                            max-width: 800px;
                            margin: 0 auto;
                        }}

                        h1 {{
                            color: #667eea;
                            border-bottom: 3px solid #667eea;
                            padding-bottom: 10px;
                            margin-top: 30px;
                        }}

                        h2 {{
                            color: #764ba2;
                            margin-top: 25px;
                        }}

                        h3 {{
                            color: #555;
                            margin-top: 20px;
                        }}

                        blockquote {{
                            background: #f8f9fa;
                            border-left: 4px solid #667eea;
                            padding: 15px;
                            margin: 20px 0;
                        }}

                        ul, ol {{
                            margin-left: 20px;
                        }}

                        li {{
                            margin: 8px 0;
                        }}

                        strong {{
                            color: #667eea;
                        }}

                        hr {{
                            border: none;
                            border-top: 2px solid #ddd;
                            margin: 30px 0;
                        }}
                    </style>
                </head>
                <body>
                    {html_content}
                </body>
                </html>
                """

                # 生成 PDF
                pdf_filename = f"{country}_{city}_{num_days}日遊行程.pdf"
                pdf_path = f"/tmp/{pdf_filename}"

                HTML(string=html_full).write_pdf(pdf_path)

                print(f"✅ PDF 已生成: {pdf_path}")

            except Exception as e:
                print(f"❌ PDF 生成失敗: {e}")
                pdf_path = None

        return full_itinerary, pdf_path

    except Exception as e:
        return f"❌ 行程生成失敗: {str(e)}", None

In [38]:
def generate_itinerary_amy(country, city, start_date, num_days, mbti, budget, interests):
    """行程生成功能"""

    # 檢查模型
    if not model:
        return "❌ Gemini API 未配置，請設定 API Key", None

    try:
        # 取得 MBTI 特徵
        mbti_info = MBTI_DESCRIPTIONS.get(mbti, {"traits": "均衡", "interests": "多元"})

        # 取得天氣資訊
        weather_info = get_weather_info(country, city, start_date, num_days)

        # 取得匯率資訊
        exchange_text = format_exchange_info(country, budget)

        # 建構 prompt
        prompt = f"""
你是專業的旅遊規劃師。請為以下需求規劃 {num_days} 天的詳細行程：

**旅遊資訊**
- 目的地：{country} {city}
- 出發日期：{start_date}
- 天數：{num_days} 天
- MBTI 人格：{mbti}（{mbti_info['traits']}）
- 預算：{budget}
- 興趣：{interests}

**天氣預報**
{json.dumps(weather_info, ensure_ascii=False, indent=2)}

{exchange_text if exchange_text else ""}

請提供：
1. 每日行程（包含景點、活動、餐廳）
2. 根據 MBTI 特質客製化推薦
3. 預算分配建議（參考上述匯率資訊）
4. 交通方式建議
5. 注意事項

請用 Markdown 格式輸出。
"""

        # 呼叫 Gemini API
        response = model.generate_content(
            prompt,
            safety_settings={
                HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
                HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
                HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
                HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
            },
        )

        itinerary_text = response.text

        # 生成 PDF
        pdf_path = None
        if WEASYPRINT_AVAILABLE:
            try:
                html_content = markdown.markdown(itinerary_text)

                html_full = f"""
                <html>
                <head>
                    <meta charset="utf-8">
                    <style>
                        body {{ font-family: "Noto Sans TC", sans-serif; line-height: 1.6; padding: 20px; }}
                        h1 {{ color: #2c3e50; }}
                        h2 {{ color: #3498db; }}
                    </style>
                </head>
                <body>{html_content}</body>
                </html>
                """

                pdf_path = f"/tmp/{country}_{city}_行程.pdf"
                HTML(string=html_full).write_pdf(pdf_path)

            except Exception as e:
                print(f"PDF 生成失敗: {e}")

        return itinerary_text, pdf_path

    except Exception as e:
        return f"❌ 生成失敗: {str(e)}", None


## ⭐ 功能 2: 行程優化器

上傳並優化現有行程：
- 📤 PDF/TXT 檔案上傳
- 🎯 5 種旅行風格選擇
- ⏰ 3 種節奏調整
- 📄 優化後 PDF 匯出

In [39]:
# 4. 組員2的功能 - 行程優化器

import re
import requests

pacing_data = {
    'planner': {'title': '⚡ 效率 (充實)', 'prompt_hint': '每天5-6個景點，緊湊', 'desc': '分秒必爭，打卡最大化'},
    'balanced': {'title': '⚖️ 平衡 (質感)', 'prompt_hint': '每天3-4個景點', 'desc': '景點與休息並重，享受漫遊樂趣'},
    'wanderer': {'title': '☕ 慢活 (度假)', 'prompt_hint': '每天2-3個景點', 'desc': '睡到自然醒，以放鬆體驗為主'},
}
pacing_choices = [info['title'] for info in pacing_data.values()]

style_data = {
    'luxury':   {'title': '🏙️ 奢華', 'desc': '五星級住宿與米其林饗宴'},
    'nature':   {'title': '🌲 秘境', 'desc': '深入山林海濱，遠離塵囂'},
    'history':  {'title': '⛩️ 古蹟', 'desc': '漫步老街，品味歷史文化'},
    'shopping': {'title': '🛍️ 血拼', 'desc': '鎖定商圈，盡情享受購物'},
    'food':     {'title': '🍽️ 美食', 'desc': '用味蕾地圖探索這座城市'},
}
style_choices = [info['title'] for info in style_data.values()]

# 風格映射字典
style_map = {v['title']: v['desc'] for v in style_data.values()}


# PDF 轉檔函數
def convert_markdown_to_pdf_weasy(markdown_content, filename="優化後行程.pdf"):
    """將 Markdown 轉換為 PDF，支援中文字體和表格"""
    FONT_FILENAME = "NotoSansTC-Regular.ttf"
    if not WEASYPRINT_AVAILABLE:
        return None

    try:
        html_body = markdown.markdown(markdown_content, extensions=['tables'])
        font_face_css = ""
        body_font_family = "sans-serif"

        # 檢查中文字體
        if os.path.exists(FONT_FILENAME):
            abspath = os.path.abspath(FONT_FILENAME)
            font_face_css = f"""@font-face {{ font-family: 'NotoSansTC'; src: url('file://{abspath}'); }}"""
            body_font_family = "'NotoSansTC', sans-serif"

        html_content = f"""
        <!DOCTYPE html>
        <html><head><meta charset="utf-8"><style>
        {font_face_css}
        @page {{ margin: 2cm; }}
        body {{ font-family: {body_font_family}; background: #fff; color: #000 !important; font-size: 10pt; line-height: 1.5; }}
        h1, h2, h3, p, li, td, th {{ color: #000000 !important; }}
        table {{ width: 100%; border-collapse: collapse; margin-top: 15px; border: 1px solid #000; }}
        th, td {{ border: 1px solid #000; padding: 8px; text-align: left; vertical-align: top; }}
        th {{ background: #EEE; font-weight: bold; }}
        </style></head><body><h1>TripStarter 行程優化報告</h1>{html_body}</body></html>
        """

        font_config = FontConfiguration()
        HTML(string=html_content).write_pdf(filename, font_config=font_config)
        return filename
    except Exception as e:
        print(f"PDF 生成失敗: {e}")
        return None


# 天氣資料顯示 HTML
def get_weather_animation_html(city_name, weather_data):
    """生成天氣預報的 HTML 卡片"""
    if not weather_data:
        return "<div class='inset-box'>⚠️ 天氣數據無法取得</div>"

    css = """<style>
    .w-card{min-width:80px;background:#fff;border:1px solid #000;margin-right:12px;color:#000;display:inline-block;text-align:center;padding:12px;}
    .w-scroll{white-space:nowrap;overflow-x:auto;padding:10px;}
    </style>"""

    cards = ""
    try:
        daily = weather_data.get('daily', {})
        for i in range(min(7, len(daily.get('time', [])))):
            date = daily['time'][i][5:].replace("-", "/")
            temp = daily['temperature_2m_max'][i]
            code = daily['weathercode'][i]
            icon = "☀️" if code in [0,1] else "🌧️" if code>45 else "☁️"
            cards += f"<div class='w-card'><div style='font-size:12px;color:#000;'>{date}</div><div style='font-size:24px;margin:5px 0;'>{icon}</div><div style='font-weight:700;color:#000;'>{temp}°</div></div>"
    except:
        return "<div class='inset-box'>數據解析錯誤</div>"

    return css + f"<div><div style='font-weight:700;color:#000;margin-bottom:8px;font-size:1.1rem;'>🏙️ {city_name} FORECAST</div><div class='w-scroll'>{cards}</div></div>"


# 動態描述更新
def update_pacing_desc(title):
    """根據選擇的節奏更新描述"""
    for info in pacing_data.values():
        if info['title'] == title:
            return f"<div style='color:#000;font-size:0.9rem;margin-top:5px;padding:5px;'>💡 {info['desc']}</div>"
    return ""


def update_style_desc(title):
    """根據選擇的風格更新描述"""
    desc = style_map.get(title, "")
    return f"<div style='color:#000;font-size:0.9rem;margin-top:5px;padding:5px;'>💡 {desc}</div>"


# AI 呼叫函數
def call_ai(prompt):
    """統一的 AI 呼叫接口"""
    global model
    if not model:
        return "⚠️ Gemini API 未配置"

    try:
        response = model.generate_content(prompt)
        return response.text
    except Exception as e:
        return f"⚠️ AI 呼叫失敗: {e}"


# 主要優化函數
def optimize_itinerary(file, pacing, styles, additional_requests):
    """行程優化功能 - 整合天氣、PDF 輸出等完整功能"""

    if not model:
        return "❌ Gemini API 未配置，請設定 API Key", None, None, None

    if not file:
        return "❌ 請上傳行程檔案", None, None, None

    try:
        # 讀取檔案內容
        if file.name.endswith('.pdf'):
            if WEASYPRINT_AVAILABLE:
                reader = PdfReader(file.name)
                original_text = "\n".join([page.extract_text() for page in reader.pages])
            else:
                return "❌ PDF 讀取功能不可用", None, None, None
        else:
            original_text = file.read().decode('utf-8')

        # 提取城市名稱 (用於天氣查詢)
        city = "Taipei"
        weather_data = None
        weather_summary = "無天氣資訊"

        try:
            city_res = call_ai("Extract only the main city name (English) from this text: " + original_text[:500])
            clean_city = re.sub(r'[^a-zA-Z\s]', '', city_res).strip()
            if clean_city and len(clean_city) < 30:
                city = clean_city

            # 查詢天氣
            r = requests.get(
                f"https://geocoding-api.open-meteo.com/v1/search?name={city}&count=1&language=zh&format=json",
                timeout=5
            )
            if r.ok and r.json().get("results"):
                res0 = r.json()["results"][0]
                lat, lon = res0["latitude"], res0["longitude"]
                weather_data = requests.get(
                    f"https://api.open-meteo.com/v1/forecast?latitude={lat}&longitude={lon}&daily=weathercode,temperature_2m_max&timezone=auto",
                    timeout=5
                ).json()

                if 'daily' in weather_data:
                    temps = weather_data['daily'].get('temperature_2m_max', [])
                    avg_t = sum(temps)/len(temps) if temps else 25
                    weather_summary = f"平均氣溫 {avg_t:.1f}°C"
        except Exception as e:
            print(f"天氣查詢失敗: {e}")

        # 節奏描述
        pacing_hint = next(
            (v['prompt_hint'] for v in pacing_data.values() if v['title'] == pacing),
            ''
        )

        # 風格描述
        style_hints = [
            style_data[k]['desc']
            for k in style_data.keys()
            if style_data[k]['title'] in styles
        ]

        # 建構詳細 prompt
        prompt = f"""
你是專業的旅遊規劃優化師。請根據以下需求優化行程：

**原始行程**
{original_text}

**優化需求**
- 節奏：{pacing} - {pacing_hint}
- 風格：{", ".join(styles)}
{chr(10).join(['  * ' + hint for hint in style_hints])}
- 天氣資訊：{weather_summary}
- 額外需求：{additional_requests if additional_requests else "無"}

**輸出格式要求**
1. **行程總覽** (含穿搭建議、必備物品)
2. **每日詳細行程** (使用 Markdown 表格)
   - 表格欄位：時間 | 地點 | 細項特色 | 交通方式 | 美食推薦 | 預算
   - 每個景點必須列出具體看點
   - 交通方式請具體說明 (如：捷運紅線、公車123號)
   - 美食推薦至少5項，包含價格範圍
   - 標註單項預算

請使用純黑體字，繁體中文輸出。保持原有結構但調整內容以符合需求。
"""

        # 呼叫 AI
        optimized_text = call_ai(prompt)

        # 生成 PDF
        pdf_path = None
        if WEASYPRINT_AVAILABLE:
            pdf_path = "/tmp/優化後行程.pdf"
            pdf_result = convert_markdown_to_pdf_weasy(optimized_text, pdf_path)
            if not pdf_result:
                pdf_path = None

        # 生成天氣 HTML
        weather_html = get_weather_animation_html(city, weather_data)

        return optimized_text, pdf_path, weather_html, "<hr style='border-top:2px solid #000;'>"

    except Exception as e:
        return f"❌ 優化失敗: {str(e)}", None, None, None

## ⭐ 功能 3: 地點推薦器

搜尋並推薦附近景點：
- 🔍 SerpAPI 真實地點搜尋
- 📍 地理定位與距離計算
- 🤖 AI 智慧推薦分析
- 🗺️ Folium 互動式地圖
- 📊 評分排序表格

In [40]:
# ==================== 組員3的功能 - 地點推薦器(完整版) ====================

import json
import pandas as pd
import folium
from math import radians, sin, cos, sqrt, atan2

# 動態檢查套件是否可用
try:
    from serpapi import GoogleSearch
    SERPAPI_AVAILABLE = True
except ImportError:
    SERPAPI_AVAILABLE = False
    print("⚠️ serpapi 套件未安裝,請執行: !pip install google-search-results")

try:
    from geopy.geocoders import Nominatim
    GEOPY_AVAILABLE = True
except ImportError:
    GEOPY_AVAILABLE = False
    print("⚠️ geopy 套件未安裝,請執行: !pip install geopy")

# ==================== 工具函數 ====================

def haversine(lat1, lon1, lat2, lon2):
    """計算兩點間距離(公尺)"""
    R = 6371000  # 地球半徑(公尺)

    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c


def get_location_coordinates(country, city, landmark):
    """獲取地點座標(使用 geopy),改進中文支援"""

    if not GEOPY_AVAILABLE:
        return None, None, "❌ Geopy 套件未安裝"

    try:
        geolocator = Nominatim(user_agent="travel_planner")

        # 多種查詢策略
        queries = []

        if landmark:
            queries.append(f"{landmark}, {city}, {country}")
            queries.append(f"{landmark}, {city}")
            queries.append(f"{landmark}")
        else:
            queries.append(f"{city}, {country}")

        # 嘗試所有查詢策略
        for query in queries:
            try:
                location = geolocator.geocode(query, timeout=10, language='zh-TW')
                if location:
                    display_name = location.address[:60] + "..." if len(location.address) > 60 else location.address
                    return location.latitude, location.longitude, f"✅ 定位成功: {display_name}"
            except:
                continue

        return None, None, f"❌ 無法解析地點: {landmark or city}"

    except Exception as e:
        return None, None, f"❌ 座標獲取錯誤: {str(e)}"


def radius_to_zoom(radius_km):
    """根據搜尋半徑轉換地圖縮放層級"""
    if radius_km <= 1:
        return 16
    elif radius_km <= 2:
        return 15
    elif radius_km <= 5:
        return 14
    elif radius_km <= 10:
        return 13
    else:
        return 12


def create_map_html(lat, lng, location_name, top_places_df=None, radius_km=2):
    """建立 Folium 地圖並回傳 HTML 字串 - 支援標註前三名餐廳"""

    if lat is None or lng is None:
        return "<div style='height: 400px; display: flex; align-items: center; justify-content: center; color: red; font-size: 16px; background-color: #fff3cd; border: 2px dashed #ffc107; border-radius: 8px;'>⚠️ 無法解析地點座標,地圖無法顯示</div>"

    try:
        # 根據半徑調整地圖縮放層級
        zoom_level = radius_to_zoom(radius_km)

        # 建立地圖
        m = folium.Map(location=[lat, lng], zoom_start=zoom_level)

        # 加入原始地標標記 (紅色)
        folium.Marker(
            [lat, lng],
            tooltip=location_name,
            popup=f"📍 {location_name}",
            icon=folium.Icon(color='red', icon='info-sign')
        ).add_to(m)

        # 標註前三名餐廳
        if top_places_df is not None and not top_places_df.empty:

            # 定義前三名的顏色和圖標
            medal_icons = [
                {'color': 'orange', 'icon': 'star', 'prefix': 'fa', 'medal': '🥇'},  # 第一名
                {'color': 'lightblue', 'icon': 'star', 'prefix': 'fa', 'medal': '🥈'},  # 第二名
                {'color': 'lightgreen', 'icon': 'star', 'prefix': 'fa', 'medal': '🥉'}   # 第三名
            ]

            for idx, (row_idx, row) in enumerate(top_places_df.iterrows()):
                # 檢查是否有座標
                if pd.notna(row.get('latitude')) and pd.notna(row.get('longitude')):
                    place_lat = row['latitude']
                    place_lng = row['longitude']

                    # 決定要用哪個圖標 (前三名)
                    if idx < 3:
                        medal_info = medal_icons[idx]

                        # 建立彈出視窗內容
                        popup_html = f"""
                        <div style="width: 200px; font-family: Arial;">
                            <h4 style="margin: 5px 0;">{medal_info['medal']} {row['名稱']}</h4>
                            <p style="margin: 5px 0;"><b>評分:</b> ⭐ {row['評分']}</p>
                            <p style="margin: 5px 0;"><b>距離:</b> {row['距離']}</p>
                            <p style="margin: 3px 0; font-size: 11px; color: #666;">{row['地址']}</p>
                        </div>
                        """

                        # 加入標記
                        folium.Marker(
                            [place_lat, place_lng],
                            tooltip=f"{medal_info['medal']} {row['名稱']}",
                            popup=folium.Popup(popup_html, max_width=250),
                            icon=folium.Icon(
                                color=medal_info['color'],
                                icon=medal_info['icon'],
                                prefix=medal_info['prefix']
                            )
                        ).add_to(m)

        return m._repr_html_()

    except Exception as e:
        print(f"地圖生成失敗: {e}")
        return f"<div style='height: 400px; display: flex; align-items: center; justify-content: center; color: orange;'>⚠️ 地圖生成錯誤: {str(e)}</div>"


def search_nearby_places(country, city, landmark, place_type, user_lat=None, user_lng=None, radius_km=2):
    """搜尋附近地點 - 完全照範例邏輯,保存座標資訊"""

    # 檢查 serpapi 是否可用
    if not SERPAPI_AVAILABLE:
        return pd.DataFrame({
            "名稱": ["⚠️ serpapi 套件未安裝"],
            "評分": [0],
            "地址": ["請執行: !pip install google-search-results"],
            "距離": ["N/A"],
            "latitude": [None],
            "longitude": [None]
        })

    # 若沒有 SerpAPI Key,返回範例資料
    if not SERPAPI_KEY or len(SERPAPI_KEY) < 10:
        return pd.DataFrame({
            "名稱": ["⚠️ 請設定 SerpAPI Key"],
            "評分": [0],
            "地址": ["無法搜尋附近地點"],
            "距離": ["N/A"],
            "latitude": [None],
            "longitude": [None]
        })

    # 若沒有座標,無法搜尋
    if user_lat is None or user_lng is None:
        return pd.DataFrame({
            "名稱": ["無法搜尋"],
            "評分": [0],
            "地址": ["座標解析失敗"],
            "距離": ["N/A"],
            "latitude": [None],
            "longitude": [None]
        })

    try:
        # 中文類型轉英文
        translation_map = {
            "餐廳": "restaurants",
            "咖啡廳": "cafes",
            "飲料店": "bubble tea shop",
            "景點": "tourist attractions",
            "住宿": "hotels",
            "寺廟": "temples",
            "百貨公司": "department store",
        }

        search_query = translation_map.get(place_type, place_type)

        print(f"🔍 搜尋查詢: {search_query}")
        print(f"📍 座標: ({user_lat}, {user_lng})")
        print(f"🔍 搜尋半徑: {radius_km} 公里")

        # 計算縮放層級
        zoom_level = radius_to_zoom(radius_km)

        # 簡潔的參數設定 - 只用類型 + 座標
        params = {
            "engine": "google_maps",
            "q": search_query,  # 只有英文類型
            "ll": f"@{user_lat},{user_lng},{zoom_level}z",  # 座標定位
            "type": "search",
            "api_key": SERPAPI_KEY,
            "hl": "en"  # 英文介面
        }

        # 使用 GoogleSearch 類
        search = GoogleSearch(params)
        results = search.get_dict()

        print(f"📦 API 回應鍵: {list(results.keys())}")

        local_results = results.get("local_results", [])[:10]  # 取前 10 筆

        if not local_results:
            print("⚠️ 無 local_results")
            return pd.DataFrame({
                "名稱": ["查無結果"],
                "評分": [0],
                "地址": [f"查詢: {search_query}"],
                "距離": ["N/A"],
                "latitude": [None],
                "longitude": [None]
            })

        print(f"✅ 找到 {len(local_results)} 個結果")

        data = []
        for r in local_results:
            name = r.get("title", "未知")
            rating = r.get("rating", 0)
            address = r.get("address", "地址不詳")
            gps = r.get("gps_coordinates", {})
            lat, lng = gps.get("latitude"), gps.get("longitude")

            # 計算距離
            distance = "N/A"
            if lat and lng:
                d = haversine(user_lat, user_lng, lat, lng)
                distance = f"{d/1000:.2f} 公里" if d > 1000 else f"{d:.0f} 公尺"

            data.append({
                "名稱": name,
                "評分": rating if rating else "無評分",
                "地址": address,
                "距離": distance,
                "latitude": lat,  # 保存緯度
                "longitude": lng  # 保存經度
            })

        return pd.DataFrame(data)

    except Exception as e:
        print(f"❌ 搜尋失敗: {e}")
        import traceback
        traceback.print_exc()

        return pd.DataFrame({
            "名稱": ["搜尋錯誤"],
            "評分": [0],
            "地址": [str(e)[:100]],
            "距離": ["N/A"],
            "latitude": [None],
            "longitude": [None]
        })


# ==================== 主要推薦函數 ====================

def recommend_places(country, city, landmark, place_type, radius_km=2):
    """
    組員3的地點推薦功能 - 支援半徑調整與前三名標註

    參數:
        country: 國家
        city: 城市
        landmark: 地標
        place_type: 地點類型
        radius_km: 搜尋半徑(公里), 預設2

    回傳:
        (DataFrame, str, str): 搜尋結果表格, AI推薦文字, 地圖HTML
    """

    if not GEOPY_AVAILABLE:
        empty_df = pd.DataFrame({"名稱": ["系統錯誤"], "評分": [0], "地址": ["Geopy 未安裝"], "距離": ["N/A"]})
        return empty_df, "❌ 地理位置功能不可用", "<div style='color: red;'>請安裝 geopy 套件</div>"

    try:
        # 1️⃣ 獲取座標
        lat, lng, geo_message = get_location_coordinates(country, city, landmark)

        print(f"📍 座標結果: lat={lat}, lng={lng}")
        print(f"📍 訊息: {geo_message}")

        # 2️⃣ 搜尋附近地點
        df = search_nearby_places(country, city, landmark, place_type, lat, lng, radius_km)

        # 檢查是否有有效結果
        if df.empty or (len(df) == 1 and df.iloc[0]["名稱"] in ["查無結果", "搜尋錯誤", "無法搜尋", "⚠️ serpapi 套件未安裝"]):
            location_display = f"{landmark}, {city}" if landmark else city
            map_html = create_map_html(lat, lng, location_display, None, radius_km)

            error_msg = f"""
❌ **搜尋失敗**

{geo_message}

**可能原因:**
1. 該區域缺少 {place_type} 的資料
2. SerpAPI 配額用完或 API Key 無效
3. 搜尋半徑 {radius_km} 公里內無結果
4. serpapi 套件未安裝

**建議:**
- 嘗試擴大搜尋半徑
- 更換其他地點或類型
- 檢查 API Key 是否有效
- 確認已安裝: !pip install google-search-results
"""
            return df[['名稱', '評分', '地址', '距離']] if '名稱' in df.columns else df, error_msg, map_html

        # 篩選前三名(依評分排序)
        df_sorted = df.copy()
        df_sorted['評分_數值'] = pd.to_numeric(df_sorted['評分'], errors='coerce')
        df_sorted = df_sorted.sort_values('評分_數值', ascending=False)
        top_3_places = df_sorted.head(3)

        print(f"🏆 前三名: {list(top_3_places['名稱'])}")

        # 3️⃣ AI 推薦
        recommendation = ""
        if gemini_client and not df.empty:
            try:
                valid_df = df_sorted[df_sorted['評分_數值'] > 0]

                if len(valid_df) >= 1:
                    top_df = valid_df.head(3)
                    # 準備給 AI 的資料(不包含座標)
                    data_list = top_df[['名稱', '評分', '地址', '距離']].to_dict(orient="records")

                    location_desc = f"{landmark}, {city}" if landmark else city

                    prompt = f"""
你是美食推薦顧問。這是 {location_desc} 附近 {radius_km} 公里內評分最高的 {place_type}:
{json.dumps(data_list, ensure_ascii=False, indent=2)}

請用 100-150 字中文,簡潔推薦這些地點的特色。
"""

                    response = model.generate_content(prompt)
                    recommendation = response.text.strip()
                else:
                    recommendation = "⚠️ 搜尋結果中沒有評分資料,請參考下方表格。"

            except Exception as e:
                if "429" in str(e) or "quota" in str(e).lower():
                    recommendation = """
⚠️ **Gemini API 配額已用完**

請查看下方搜尋結果表格,根據評分和距離自行選擇。
"""
                else:
                    recommendation = f"⚠️ AI 分析失敗: {str(e)[:100]}\n\n請參考下方搜尋結果。"
        else:
            recommendation = "⚠️ Gemini API 未設定,請參考下方搜尋結果。"

        # 4️⃣ 生成地圖 - 傳入前三名資料與半徑
        location_display = f"{landmark}, {city}" if landmark else city
        map_html = create_map_html(lat, lng, location_display, top_3_places, radius_km)

        # 5️⃣ 回傳給使用者的 DataFrame(移除座標欄位)
        display_df = df_sorted[['名稱', '評分', '地址', '距離']].copy()

        return display_df, recommendation, map_html

    except Exception as e:
        import traceback
        traceback.print_exc()

        empty_df = pd.DataFrame({"名稱": ["系統錯誤"], "評分": [0], "地址": [str(e)[:100]], "距離": ["N/A"]})
        return empty_df, f"❌ 推薦失敗: {str(e)}", "<div style='color: red;'>系統錯誤</div>"

## 🎨 步驟 6: 建立 Gradio 介面

整合所有功能到統一介面

In [41]:
# 6. Gradio 介面
# ============================================

# CSS 樣式
css = """
.container {
    max-width: 1400px;
    margin: 0 auto;
    padding: 20px;
}

.main-title {
    font-size: 48px;
    font-weight: bold;
    text-align: center;
    background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
    -webkit-background-clip: text;
    -webkit-text-fill-color: transparent;
    margin-bottom: 10px;
    letter-spacing: 3px;
}

.sub-title {
    text-align: center;
    color: #6c757d;
    font-size: 18px;
    margin-bottom: 40px;
    letter-spacing: 2px;
}

.section-header {
    font-size: 20px;
    font-weight: 600;
    color: #495057;
    margin: 20px 0 10px 0;
    padding-bottom: 10px;
    border-bottom: 2px solid #dee2e6;
}

.panel-container {
    background: white;
    border-radius: 12px;
    padding: 20px;
    box-shadow: 0 2px 8px rgba(0,0,0,0.1);
}

.gradio-container {
    background: linear-gradient(135deg, #f5f7fa 0%, #c3cfe2 100%);
}
"""

# Gradio 介面
with gr.Blocks(css=css, title="TRIP STARTER") as demo:

    with gr.Column(elem_classes="container"):

        gr.HTML("""
        <div class='main-title'>TRIP STARTER</div>
        <div class='sub-title'>Intelligent Travel Planner</div>
        """)

        with gr.Tabs():
            # ==========================================
            # TAB 1: 旅客類型問卷
            # ==========================================
            with gr.Tab("📋 旅客類型分析"):

                gr.HTML("""
                <div class='questionnaire-hint'>
                    <strong>💡 為什麼需要問卷?</strong><br>
                    透過簡單5題問卷,我們將分析你的旅遊偏好,為你量身打造最適合的行程規劃!
                </div>
                """)

                with gr.Row():
                    with gr.Column(scale=1, elem_classes="panel-container"):

                        gr.HTML("<div class='section-header'>📝 請回答以下問題</div>")

                        # Q1: 旅遊期待 (多選)
                        q1_input = gr.CheckboxGroup(
                            choices=[
                                "刺激冒險體驗",
                                "深度文化探索",
                                "品嚐在地美食",
                                "血拼購物",
                                "放鬆身心",
                                "親近大自然",
                                "體驗夜生活",
                                "親子同樂"
                            ],
                            label="1️⃣ 旅遊時你最期待什麼? (可複選,最多3個)",
                            info="請選擇1-3個最符合你的選項"
                        )

                        # Q2: 行程節奏
                        q2_input = gr.Radio(
                            choices=[
                                "緊湊充實,每天多個景點",
                                "適中平衡,有景點也有休息",
                                "彈性自由,隨興安排",
                                "緩慢悠閒,深度停留"
                            ],
                            label="2️⃣ 你偏好的行程節奏是?",
                            value="適中平衡,有景點也有休息"
                        )

                        # Q3: 旅伴類型
                        q3_input = gr.Radio(
                            choices=[
                                "獨自旅行",
                                "情侶/夫妻",
                                "家庭親子",
                                "朋友團體"
                            ],
                            label="3️⃣ 同行旅伴類型?",
                            value="朋友團體"
                        )

                        # Q4: 活動強度
                        q4_input = gr.Radio(
                            choices=[
                                "高強度(登山、極限運動等)",
                                "中等強度(健行、單車等)",
                                "輕度活動(散步、參觀等)",
                                "低強度(度假、休息為主)"
                            ],
                            label="4️⃣ 你喜歡的活動強度?",
                            value="中等強度(健行、單車等)"
                        )

                        # Q5: 預算分配 (多選)
                        q5_input = gr.CheckboxGroup(
                            choices=[
                                "特色活動體驗",
                                "高級餐廳美食",
                                "購物與紀念品",
                                "舒適住宿",
                                "交通移動",
                                "夜間娛樂",
                                "親子設施門票"
                            ],
                            label="5️⃣ 旅遊預算主要想花在哪? (選2個)",
                            info="請選擇2個最重要的項目"
                        )

                        analyze_btn = gr.Button("🔍 分析我的旅客類型", variant="primary", size="lg")

                    # 右側: 分析結果
                    with gr.Column(scale=1, elem_classes="panel-container"):
                        gr.HTML("<div class='section-header'>✨ 分析結果</div>")

                        type_result = gr.Markdown(
                            value="請完成左側問卷後,點擊「分析我的旅客類型」按鈕",
                            label="你的旅客類型"
                        )

                        # 隱藏的狀態變數,儲存分析結果
                        analysis_state = gr.State(value=None)


                analyze_btn.click(
                    fn=process_questionnaire,
                    inputs=[q1_input, q2_input, q3_input, q4_input, q5_input],
                    outputs=[type_result, analysis_state]
                )

            # -------------------------------------------------------
            # TAB 2：自動生成行程
            # -------------------------------------------------------
            with gr.Tab("✨ 自動生成行程"):

                gr.HTML("""
                <div class='questionnaire-hint'>
                    <strong>💡 提示:</strong> 建議先完成「旅客類型分析」,系統會自動根據你的類型客製化行程!
                </div>
                """)

                with gr.Row():

                    # 左側表單
                    with gr.Column(scale=1, elem_classes="panel-container"):

                        gr.HTML("<div class='section-header'>🗺️ 行程基本資訊</div>")

                        country1 = gr.Textbox(
                            label="國家",
                            value="日本",
                            placeholder="例如: 日本"
                        )

                        city1 = gr.Textbox(
                            label="城市",
                            value="東京",
                            placeholder="例如: 東京"
                        )

                        start_date1 = gr.Textbox(
                            label="出發日期",
                            value="2025-06-01",
                            placeholder="YYYY-MM-DD"
                        )

                        num_days1 = gr.Slider(
                            minimum=1,
                            maximum=14,
                            value=5,
                            step=1,
                            label="旅遊天數"
                        )

                        gr.HTML("<div class='section-header'>🎯 個人偏好</div>")

                        # 🆕 改用旅客類型
                        traveler_type1 = gr.Dropdown(
                            choices=list(TRAVELER_TYPES.keys()),
                            value="美食型",
                            label="旅客類型",
                            info="建議先完成問卷,系統會自動填入"
                        )

                        budget1 = gr.Radio(
                            choices=["經濟實惠", "中等預算", "高端奢華"],
                            value="中等預算",
                            label="預算範圍"
                        )

                        interests1 = gr.Textbox(
                            label="額外興趣與偏好 (選填)",
                            placeholder="例如: 想看櫻花、喜歡動漫、對歷史特別有興趣",
                            lines=2
                        )

                        generate_btn1 = gr.Button("🚀 生成專屬行程", variant="primary", size="lg")

                    # 右側輸出
                    with gr.Column(scale=2, elem_classes="panel-container"):

                        gr.HTML("<div class='section-header'>📄 生成結果</div>")

                        itinerary_output1 = gr.Markdown(
                            value="點擊左側「生成專屬行程」開始規劃你的旅程!",
                            label="行程內容"
                        )

                        pdf_output1 = gr.File(
                            label="📥 下載 PDF 版本",
                            visible=True
                        )

                # 🆕 綁定新的生成函數
                generate_btn1.click(
                    fn=generate_itinerary,
                    inputs=[
                        country1,
                        city1,
                        start_date1,
                        num_days1,
                        traveler_type1,
                        budget1,
                        interests1
                    ],
                    outputs=[itinerary_output1, pdf_output1]
                )

                # 🆕 當問卷分析完成後,自動更新旅客類型
                def update_traveler_type(analysis):
                    """從問卷分析結果更新旅客類型"""
                    if analysis and "primary_type" in analysis:
                        return analysis["primary_type"]
                    return "美食型"

                # 監聽問卷分析結果
                analysis_state.change(
                    fn=update_traveler_type,
                    inputs=[analysis_state],
                    outputs=[traveler_type1]
                )

            # -------------------------------------------------------
            # TAB 3：行程優化器
            # -------------------------------------------------------
            with gr.Tab("🔧 行程優化器"):

                with gr.Row(equal_height=False):

                    # 左側控制台
                    with gr.Column(scale=1, elem_classes="panel-container"):

                        gr.HTML("<div class='section-header'>1. 行程檔案</div>")
                        file_input2 = gr.File(label="", file_count="single", type="filepath")

                        gr.HTML("<div class='section-header' style='margin-top:15px;'>2. 優化設定</div>")

                        # 旅遊節奏
                        pacing2 = gr.Radio(
                            choices=pacing_choices,
                            value=pacing_choices[1],
                            label="旅遊節奏",
                            show_label=True,
                            elem_classes="pacing-choices"
                        )
                        pacing_desc2 = gr.HTML(update_pacing_desc(pacing_choices[1]))

                        gr.HTML("<div style='height:10px'></div>")  # 縮小間距: 20px → 10px

                        # 風格偏好
                        gr.HTML("<div class='section-header' style='margin-top:5px;'>3. 風格偏好</div>")
                        styles2 = gr.CheckboxGroup(
                            choices=style_choices,
                            label="",
                            show_label=False,
                            elem_classes="style-choices"
                        )

                        gr.HTML("<div style='height:10px'></div>")  # 縮小間距: 20px → 10px

                        # 額外需求
                        gr.HTML("<div class='section-header' style='margin-top:5px;'>4. 額外需求</div>")
                        additional2 = gr.Textbox(
                            label="",
                            placeholder="例如：想加入更多美食 / 想避開人潮",
                            lines=3,
                            show_label=False
                        )

                        gr.HTML("<div style='height:20px'></div>")  # 縮小間距: 30px → 20px
                        optimize_btn2 = gr.Button("開始優化行程 🛠️", variant="primary", elem_classes="primary")

                    # 右側結果顯示
                    with gr.Column(scale=2, elem_classes="panel-container"):

                        weather_html2 = gr.HTML("<div class='inset-box'>☁️ 等待分析...</div>")
                        divider2 = gr.HTML("<hr style='border:0; border-top:2px solid #000; margin:25px 0;'>")

                        gr.HTML("<div style='font-weight:700; margin-bottom:15px; color:#000000; font-size:1.2rem; text-transform:uppercase;'>優化後行程</div>")

                        optimized_output2 = gr.Markdown("優化結果將顯示於此...", elem_classes="result-area")

                        # ⭐ 關鍵修正:移除所有 visible=False
                        optimized_pdf2 = gr.File(label="📥 下載優化後 PDF")

                    # 綁定事件
                    optimize_btn2.click(
                        fn=optimize_itinerary,
                        inputs=[file_input2, pacing2, styles2, additional2],
                        outputs=[optimized_output2, optimized_pdf2, weather_html2, divider2]
                    )

                    # 動態更新描述
                    pacing2.change(
                        fn=update_pacing_desc,
                        inputs=pacing2,
                        outputs=pacing_desc2
                    )


            # -------------------------------------------------------
            # TAB 4:地點推薦器
            # -------------------------------------------------------
            with gr.Tab("📍 地點推薦器"):

                gr.Markdown("### 🌍 智慧地點推薦器")
                gr.Markdown("請輸入地點與想找的類型,AI 會幫你推薦附近最佳選項。地圖會標示前三名高分地點!")

                with gr.Row():

                    # 左側搜尋條件
                    with gr.Column(scale=1, elem_classes="panel-container"):

                        gr.HTML("<div class='section-header'>搜尋條件</div>")

                        country3 = gr.Textbox(
                            label="國家",
                            value="台灣",
                            placeholder="例如:台灣、日本、韓國"
                        )

                        city3 = gr.Textbox(
                            label="城市",
                            value="台北市",
                            placeholder="例如:台北市、首爾特別市"
                        )

                        landmark3 = gr.Textbox(
                            label="地標",
                            placeholder="例如:台北101、弘大入口站"
                        )

                        place_type3 = gr.Dropdown(
                            label="搜尋類型",
                            choices=["餐廳", "咖啡廳", "飲料店", "景點", "住宿"],
                            value="餐廳"
                        )

                        # 搜尋半徑 Slider
                        radius3 = gr.Slider(
                            label="搜尋半徑(公里)",
                            minimum=0.5,
                            maximum=10,
                            step=0.5,
                            value=2,
                            info="調整搜尋範圍大小 (預設 2 公里)"
                        )

                        search_btn3 = gr.Button("🔍 開始搜尋", variant="primary")

                    # 右側結果顯示
                    with gr.Column(scale=2, elem_classes="panel-container"):

                        gr.HTML("<div class='section-header'>推薦結果</div>")

                        results_table3 = gr.Dataframe(
                            headers=["名稱", "評分", "地址", "距離"],
                            label="附近地點"
                        )

                        ai_recommendation3 = gr.Textbox(
                            label="AI 推薦",
                            lines=8,
                            interactive=False
                        )

                        map_output3 = gr.HTML(label="地圖 (🥇🥈🥉 標示前三名)")

                # 按鈕點擊事件
                search_btn3.click(
                    fn=recommend_places,
                    inputs=[country3, city3, landmark3, place_type3, radius3],
                    outputs=[results_table3, ai_recommendation3, map_output3]
                )

        # -------------------------------------------------------
        # 底部說明
        # -------------------------------------------------------
        gr.Markdown("""
        ---
        ### 💡 使用說明

        **✨ 自動生成行程**：根據目的地、日期、MBTI 人格和興趣，AI 自動生成個性化旅遊行程

        **🔧 行程優化器**：上傳現有行程，根據旅行節奏和風格偏好進行智慧優化

        **📍 地點推薦器**：搜尋指定地點附近的餐廳、景點等，並提供 AI 推薦分析

        ---
        ⚙️ *需要 Gemini API Key 才能使用 AI 功能*
        """)


/tmp/ipython-input-2888643277.py:53: DeprecationWarning: The 'css' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'css' to Blocks.launch() instead.
  with gr.Blocks(css=css, title="TRIP STARTER") as demo:


## 🚀 步驟 7: 啟動系統

執行後會出現一個連結，點擊即可開啟介面

In [42]:
# 啟動 Gradio 介面
demo.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://ef00ef9684dc0344f2.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


DEBUG:fontTools.ttLib.ttFont:Reading 'maxp' table from disk
DEBUG:fontTools.ttLib.ttFont:Decompiling 'maxp' table
DEBUG:fontTools.subset.timer:Took 0.002s to load 'maxp'
DEBUG:fontTools.subset.timer:Took 0.000s to prune 'maxp'
INFO:fontTools.subset:maxp pruned
DEBUG:fontTools.ttLib.ttFont:Reading 'cmap' table from disk
DEBUG:fontTools.ttLib.ttFont:Decompiling 'cmap' table
DEBUG:fontTools.ttLib.ttFont:Reading 'post' table from disk
DEBUG:fontTools.ttLib.ttFont:Decompiling 'post' table
DEBUG:fontTools.ttLib.ttFont:Reading 'cmap' table from disk
DEBUG:fontTools.ttLib.ttFont:Decompiling 'cmap' table
DEBUG:fontTools.subset.timer:Took 0.125s to load 'cmap'
DEBUG:fontTools.subset.timer:Took 0.000s to prune 'cmap'
INFO:fontTools.subset:cmap pruned
INFO:fontTools.subset:prep dropped
DEBUG:fontTools.subset.timer:Took 0.000s to load 'post'
DEBUG:fontTools.subset.timer:Took 0.000s to prune 'post'
INFO:fontTools.subset:post pruned
INFO:fontTools.subset:GPOS dropped
INFO:fontTools.subset:GSUB droppe

✅ PDF 已生成: /tmp/法國_巴黎_5日遊行程.pdf


DEBUG:fontTools.ttLib.ttFont:Reading 'maxp' table from disk
DEBUG:fontTools.ttLib.ttFont:Decompiling 'maxp' table
DEBUG:fontTools.subset.timer:Took 0.003s to load 'maxp'
DEBUG:fontTools.subset.timer:Took 0.000s to prune 'maxp'
INFO:fontTools.subset:maxp pruned
DEBUG:fontTools.ttLib.ttFont:Reading 'cmap' table from disk
DEBUG:fontTools.ttLib.ttFont:Decompiling 'cmap' table
DEBUG:fontTools.ttLib.ttFont:Reading 'CFF ' table from disk
DEBUG:fontTools.ttLib.ttFont:Decompiling 'CFF ' table
DEBUG:fontTools.subset.timer:Took 0.094s to load 'cmap'
DEBUG:fontTools.subset.timer:Took 0.000s to prune 'cmap'
INFO:fontTools.subset:cmap pruned
DEBUG:fontTools.ttLib.ttFont:Reading 'post' table from disk
DEBUG:fontTools.ttLib.ttFont:Decompiling 'post' table
DEBUG:fontTools.subset.timer:Took 0.002s to load 'post'
DEBUG:fontTools.subset.timer:Took 0.000s to prune 'post'
INFO:fontTools.subset:post pruned
DEBUG:fontTools.subset.timer:Took 0.000s to load 'CFF '
DEBUG:fontTools.subset.timer:Took 0.044s to pru

📍 座標結果: lat=48.8603823, lng=2.360742
📍 訊息: ✅ 定位成功: 玛莱, 巴黎, 法兰西岛大区 / 法蘭西島大區, 法國本土/法国本土, 法国
🔍 搜尋查詢: restaurants
📍 座標: (48.8603823, 2.360742)
🔍 搜尋半徑: 2 公里
📦 API 回應鍵: ['search_metadata', 'search_parameters', 'search_information', 'local_results', 'serpapi_pagination']
✅ 找到 10 個結果
🏆 前三名: ['C’est Comme À La Maison - CCALM', 'Assanabel', 'Kitchen Ter(re)']
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://ef00ef9684dc0344f2.gradio.live


---

## 📖 使用指南

### Tab 1: 自動生成行程
**輸入：**
- 國家、城市、日期、天數
- MBTI 類型（16選1）
- 預算範圍
- 興趣偏好

**輸出：**
- 📅 每日詳細行程
- 🌤️ 真實天氣預報
- 💱 即時匯率參考
- 📄 PDF 下載

---

### Tab 2: 行程優化器
**輸入：**
- 上傳行程檔案（PDF/TXT）
- 選擇節奏：輕鬆、適中、緊湊
- 選擇風格：美食、文化、自然、購物、冒險
- 額外需求（選填）

**輸出：**
- 📝 優化後行程
- 📄 PDF 下載

---

### Tab 3: 地點推薦器
**輸入：**
- 國家、城市
- 特定地標（選填）
- 搜尋類型：景點/餐廳/咖啡廳

**輸出：**
- 📊 評分排序表格
- 🤖 AI 推薦分析
- 🗺️ 互動式地圖

---

## 🐛 疑難排解

### ❓ 常見問題

**Q1: API 顯示 429 錯誤？**
```
A: 配額用完，請：
   1. 等待配額重置（每天/每月）
   2. 或更換 API Key
```

**Q2: 天氣/匯率顯示預設值？**
```
A: API Key 未設定或無效，請：
   1. 檢查 Colab Secrets 設定
   2. 確認 API Key 正確
   3. 檢查 API 配額
```

**Q3: PDF 無法生成？**
```
A: 系統套件未安裝，請：
   1. 重新執行步驟 1
   2. 確認無錯誤訊息
```

**Q4: 地點顯示範例資料？**
```
A: SerpAPI Key 未設定，請：
   1. 申請 SerpAPI Key
   2. 加入 Colab Secrets
```

**Q5: 地圖不顯示？**
```
A: 網路或套件問題，請：
   1. 檢查網路連線
   2. 重新安裝 folium
```

---

## 📚 完整文檔

更多資訊請參考：
- [系統總覽與功能說明.md](./系統總覽與功能說明.md)
- [快速設定指南.md](./快速設定指南.md)
- [API設定指南.md](./API設定指南.md)
- [Weather_API設定指南.md](./Weather_API設定指南.md)
- [匯率API設定指南.md](./匯率API設定指南.md)

---

## 🎯 API 配額管理

| API | 免費配額 | 重置週期 | 用途 |
|-----|---------|---------|------|
| Gemini | 1,500次 | 每天 | 全部功能 |
| SerpAPI | 100次 | 每月 | 地點推薦 |
| Weather | 1,000次 | 每天 | 天氣查詢 |
| Exchange | 1,500次 | 每月 | 匯率換算 |

💡 **建議**：合理使用，避免重複查詢相同內容

---

**🎉 享受你的智慧旅遊規劃體驗！**